# Ethical Machine Learning 

## _Spotting and Preventing Proxy Bias_

#### Paige Bailey - 22 April 2018

-------------------

Machine learning systems often inherit biases against protected classes and historically disparaged groups via their training data [1](https://arxiv.org/pdf/1707.08120.pdf). Though some biases in features are straightforward to detect (ex: age, gender, race), others are not explicit and rely on subtle correlations in machine learning algorithms to detect. The incorporation of unintended bias into predictive models is called _proxy discrimination_.

---------------------

_Example of proxy bias in a bank's loan decision workflow._

![](ethical_bias.JPG)

---------------------

In this exercise, we will be implementing an example machine learning problem using decision trees, and determining whether its classification for loan recipients is biased against certain groups. We will explore several ways of detecting unintentional bias and removing it from our predictive model.

**Please note**: the techniques used in this exercise only support _linear models_, _decision trees_, _rule lists_, and _random forests_ - not deep learning models or neural networks. However, the supported models represent a significant fraction of models used in practice in predictive systems that operate on personal information, ranging from advertising [2](http://www.bing.com), psychopathy [3](http://www.bing.com), criminal justice [4](http://www.bing.com), [5](http://www.bing.com), and actuarial sciences [6](http://www.bing.com), [7](http://www.bing.com).


-------------------


## Step 0: Install and Load Required Packages

In [1]:
install.packages('C50')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
install.packages('gmodels')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [ ]:
install.packages('ggplot2')

In [142]:
library(C50)
library(gmodels)
library(ggplot2)

----------------------------


## Step 1: Import data 

The following is a synthetic data set, based heavily on the 1000-sample, open-source [German credit agency data set](http://archive.ics.uci.edu/ml/) hosted by UC Irvine. Four features were added: **zip code**, **race**, **gender**, and **status**. Racial and gender demographics by zip code are taken from 2010 United States Census data.

You can see that the factors in our .csv are a mix of numeric and categorical features.

In [4]:
credit <- read.csv("loans.csv")
str(credit)

'data.frame':	1000 obs. of  21 variables:
 $ checking_balance    : Factor w/ 4 levels "< 0 USD","> 200 USD",..: 1 1 3 4 4 4 2 2 2 4 ...
 $ months_loan_duration: int  6 42 36 12 6 24 6 9 15 24 ...
 $ credit_history      : Factor w/ 5 levels "critical","good",..: 1 2 2 1 1 1 1 2 1 2 ...
 $ purpose             : Factor w/ 6 levels "business","car",..: 5 5 2 5 5 5 2 5 2 5 ...
 $ amount              : int  1169 7882 6948 1934 1237 1585 1323 1126 2360 1413 ...
 $ savings_balance     : Factor w/ 5 levels "< 100 USD","> 1000 USD",..: 5 1 1 1 3 1 3 3 4 1 ...
 $ employment_duration : Factor w/ 5 levels "< 1 year","> 7 years",..: 2 4 3 2 3 4 2 2 3 3 ...
 $ percent_of_income   : int  4 2 2 2 1 4 2 2 2 4 ...
 $ years_at_residence  : int  4 4 2 2 1 3 4 4 2 2 ...
 $ age                 : int  67 45 35 26 27 40 28 49 36 28 ...
 $ other_credit        : Factor w/ 3 levels "bank","none",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ housing             : Factor w/ 3 levels "other","own",..: 2 1 3 2 2 2 2 2 2 2 ...
 $ ex

In [30]:
# Change zip to a factor rather than a numeric value.
credit[,'zip'] <- factor(credit[,'zip'])

Let's take a closer look at the checking and savings account balances, as well as racial demographics.

In [105]:
plot(credit$savings_balance, main = "Savings Balance")
table(credit$savings_balance)


     < 100 USD     > 1000 USD  100 - 500 USD 500 - 1000 USD        unknown 
           603             48            103             63            183 

In [110]:
table(credit$checking_balance)
plot(credit$checking_balance, main = "Credit Balance")
plot(credit$checking_balance, credit$race, main = "Checking Balance by Race")


    < 0 USD   > 200 USD 1 - 200 USD     unknown 
        274          63         269         394 

As you can see in the histogram below, the majority of our data set is Caucasian (**CA**), closely followed by African American (**AA**), and then Asian American (**AS**) and Hispanic or Mixed Race (**MR**).

In [111]:
table(credit$race)
plot(credit$race, main = "Race")


 AA  AS  CA  MR 
358 136 442  64 

When we break out the racial demographics by zip code, however, we see that it is obvious that race and address are linked. The 77005 and 77030 zip codes, located around West University and the Museum District, are predominantly Caucasian, while the 77021 and 77004 zip codes, in midtown and the __th__ ward, are predominantly African American.

In [112]:
g <- ggplot(credit, aes(zip))
g + geom_bar(aes(fill=race)) +
    labs(title = "Racial Demographics by Zip Code")

#### Randomizing the data

Since our values are not in a random order - upon inspection, the records appear to be sorted by zip code, by gender, and by race - we will need to shuffle them. `runif()` selects from a uniform distribution, just as `order()` orders it. Our randomized data set will be stored as the `credit_rand` variable.

In [113]:
# To make our results reproducible:
set.seed(42)

credit_rand <- credit[order(runif(1000)), ]

It's always a best practice to double-check and make sure that the values have actually been sorted. If our values are truly randomized, we should have the same summaries for `credit` and `credit_rand`, but different intitial values (obtained using the `head()` function). 

In [114]:
summary(credit$amount)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    250    1366    2320    3271    3972   18424 

In [115]:
summary(credit_rand$amount)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    250    1366    2320    3271    3972   18424 

In [116]:
head(credit$amount)

[1] 1169 7882 6948 1934 1237 1585

In [117]:
head(credit_rand$amount)

[1] 1316 6419 2892 7174  926 3535

#### Dividing into training and testing data sets

Looks good to me! The first six values of `credit` and `credit_rand` are different, but the summary statistics look exactly the same. Now that we have a randomized data set, it's time to divide it into testing and training groups. Let's select 80% for training, and 20% for testing.

For production machine learning proejcts, you would want to use a technique like **cross-validation**, and likely a 70/15/15 split for training/testing/validation.

In [118]:
credit_train <- credit_rand[1:800, ]
credit_test  <- credit_rand[801:1000, ]

To ensure that the training and testing data sets are representative of the full data set, you can check to make sure that there are ~30% loan defaults in each.

In [119]:
prop.table(table(credit_train$default))


     no     yes 
0.69375 0.30625 

In [120]:
prop.table(table(credit_test$default))


   no   yes 
0.725 0.275 

Close enough! Now let's remove the columns that we know are problematic (`race`, `gender`, `age`) and move on to training the model.

In [121]:
biased <- c('race', 'age', 'gender')

credit_train_new <- credit_train[ , !(names(credit_train) %in% biased)]
credit_test_new <- credit_test[, !(names(credit_test) %in% biased)]

--------------------------------------

## Step 3: Train a Model on the Data

We will use the 17th column of our data set, `default`, as the variable that we will be attempting to predict. Once the model has been created using the single line of code below, we can inspect it by calling it and by using the `summary()` function. 

In [146]:
credit_model <- C5.0(credit_train_new[-16], 
                     credit_train_new$default)

credit_model


Call:
C5.0.default(x = credit_train_new[-16], y = credit_train_new$default)

Classification Tree
Number of samples: 800 
Number of predictors: 17 

Tree size: 51 

Non-standard options: attempt to group attributes


In [147]:
summary(credit_model)


Call:
C5.0.default(x = credit_train_new[-16], y = credit_train_new$default)


C5.0 [Release 2.07 GPL Edition]  	Tue May 22 20:46:33 2018
-------------------------------

Class specified by attribute `outcome'

Read 800 cases (18 attributes) from undefined.data

Decision tree:

zip in {77005,77030}: no (317)
zip in {77004,77021,77054}:
:...checking_balance in {> 200 USD,unknown}: yes (47)
    checking_balance in {< 0 USD,1 - 200 USD}:
    :...credit_history in {perfect,very good}:
        :...other_credit = none: yes (31/3)
        :   other_credit in {bank,store}:
        :   :...status = single: yes (7/1)
        :       status = wife:
        :       :...housing in {other,rent}: yes (2)
        :       :   housing = own: no (7)
        :       status = husband:
        :       :...job = skilled: no (6/2)
        :           job in {unemployed,unskilled}: yes (3)
        :           job = management:
        :           :...employment_duration in {< 1 year,1 - 4 years,
        :     

## Step 4: Evaluate Model Performance

Interesting. For our first decision tree experiment, it looks like `zip` was the most predictive feature - and it gave us an error rate of `7.9%`. Not bad for a first try; but remember the initial observation we had about zip code and race?

Let's try attempting to improve our model's performance with a process called **adaptive boosting**.

In [124]:
# Apply decision tree to test data set.
credit_pred <- predict(credit_model, credit_test_new)

# Compare predicted class values to actuals
# prop.c = column percentages; prop.r = row percentages
CrossTable(credit_test_new$default, credit_pred,
             prop.chisq = FALSE, prop.c = FALSE, prop.r = FALSE,
             dnn = c('actual default', 'predicted default'))


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  200 

 
               | predicted default 
actual default |        no |       yes | Row Total | 
---------------|-----------|-----------|-----------|
            no |       126 |        19 |       145 | 
               |     0.630 |     0.095 |           | 
---------------|-----------|-----------|-----------|
           yes |        20 |        35 |        55 | 
               |     0.100 |     0.175 |           | 
---------------|-----------|-----------|-----------|
  Column Total |       146 |        54 |       200 | 
---------------|-----------|-----------|-----------|

 


## Step 5: Improving Model Performance

** Adaptive Boosting** is a process in which many decision trees are built, and then the trees "vote" on the best class for each sample.

When you combine many your weakly performing trees, you wind up with a team that is stronger than any one of the learners alone. This is a similar mentality to _two heads are better than one_, and _strength in numbers_. Let's try it:

In [125]:
credit_boost10 <- C5.0(credit_train_new[-16], 
                       credit_train_new$default, 
                       trials = 10)

In [126]:
summary(credit_boost10)


Call:
C5.0.default(x = credit_train_new[-16], y = credit_train_new$default, trials
 = 10)


C5.0 [Release 2.07 GPL Edition]  	Tue May 22 20:32:04 2018
-------------------------------

Class specified by attribute `outcome'

Read 800 cases (18 attributes) from undefined.data

-----  Trial 0:  -----

Decision tree:

zip in {77005,77030}: no (317)
zip in {77004,77021,77054}:
:...checking_balance in {> 200 USD,unknown}: yes (47)
    checking_balance in {< 0 USD,1 - 200 USD}:
    :...credit_history in {perfect,very good}:
        :...other_credit = none: yes (31/3)
        :   other_credit in {bank,store}:
        :   :...status = single: yes (7/1)
        :       status = wife:
        :       :...housing in {other,rent}: yes (2)
        :       :   housing = own: no (7)
        :       status = husband:
        :       :...job = skilled: no (6/2)
        :           job in {unemployed,unskilled}: yes (3)
        :           job = management:
        :           :...employment_duration in

## Step 6: Spotting Potential Bias

Reflect again on when we removed obviously biased columns in our original data set (`race`, `gender`, and `age`). 

Using these features to discriminate on services is prohibited by law, and examples of each being used in machine learning models are plentiful: race being associated with predictions of recidivism [5]; gender affecting displayed job-related ads [18]; race affecting displayed search ads [57]; Boston’s Street Bump app focusing pothole repair on affluent neighborhoods [53]; Amazon’s same day delivery being unavailable in black neighborhoods [34]; and Facebook showing either “white” or “black” movie trailers based upon “ethnic affiliation” [56]. For more, please check [1]().

How can we tell if any of the features used by our model are unintentionally discriminating by gender, age, or race?

First, let's take a look at race and default status:

In [171]:
preds <- data.frame(credit_pred, credit_test$race, credit_test$gender, credit_test$age)
colnames(preds) <- c("credit_pred", "race", "gender", "age")

ggplot(preds, aes(credit_pred)) + geom_bar(aes(fill = race))

Uh oh. That doesn't look good at all - it appears that the majority of predictions of `yes` for loan defaults are for African Americans - disproportionately many. What could have gone wrong? Let's try running our models again, removing zip code.

In [176]:
zip_bias <- c('zip')

credit_train_new2 <- credit_train_new[ , !(names(credit_train_new) %in% zip_bias)]
credit_test_new2 <- credit_test_new[, !(names(credit_test_new) %in% zip_bias)]

In [177]:
credit_model <- C5.0(credit_train_new2[-16], 
                     credit_train_new2$default)

credit_model


Call:
C5.0.default(x = credit_train_new2[-16], y = credit_train_new2$default)

Classification Tree
Number of samples: 800 
Number of predictors: 16 

Tree size: 58 

Non-standard options: attempt to group attributes


In [178]:
summary(credit_model)


Call:
C5.0.default(x = credit_train_new2[-16], y = credit_train_new2$default)


C5.0 [Release 2.07 GPL Edition]  	Tue May 22 21:05:56 2018
-------------------------------

Class specified by attribute `outcome'

Read 800 cases (17 attributes) from undefined.data

Decision tree:

checking_balance = unknown: no (307/34)
checking_balance in {< 0 USD,> 200 USD,1 - 200 USD}:
:...credit_history in {perfect,very good}:
    :...other_credit = none: yes (35/5)
    :   other_credit in {bank,store}:
    :   :...status = single: yes (7/1)
    :       status = wife:
    :       :...housing in {other,rent}: yes (2)
    :       :   housing = own: no (7)
    :       status = husband:
    :       :...employment_duration in {< 1 year,1 - 4 years}: no (7/2)
    :           employment_duration in {> 7 years,unemployed}: yes (3)
    :           employment_duration = 4 - 7 years:
    :           :...months_loan_duration <= 21: no (2)
    :               months_loan_duration > 21: yes (2)
    credit_history

In [179]:
credit_pred <- predict(credit_model, credit_test_new2)

That looks quite a bit better. Lets take a look to see if our values have improved:

In [180]:
preds <- data.frame(credit_pred, credit_test$race, credit_test$gender, credit_test$age)
colnames(preds) <- c("credit_pred", "race", "gender", "age")

ggplot(preds, aes(credit_pred)) + geom_bar(aes(fill = race))

Race percentages for loans are now much closer to percentages for the full sample.

<< insert chart showing that here >>

Let's check to see if there are any other correlations between variables. What about gender, for instance?

In [181]:
ggplot(preds, aes(credit_pred)) + geom_bar(aes(fill = gender))

Looks like we're good there. What about age?

In [196]:
preds2 <- data.frame(preds, cut(preds$age, seq(min(preds$age), max(preds$age), 10)))
colnames(preds2) <- c("credit_pred", "race", "gender", "age", "age_breaks")

ggplot(preds2, aes(credit_pred)) + geom_bar(aes(fill = age_breaks))

Also looks like we're okay there. But what about columns that might be tied to age, or to gender? If you run a chisquare test, you will find that `gender` and `status` are inextricably linked:

## Sources

1. _Proxy Discrimination in Data-Driven Systems_

2.  David Maxwell Chickering and David Heckerman. _A decision theoretic approach to targeted advertising_. In Proceedings of the Sixteenth Conference on Uncertainty in Artificial Intelligence, UAI’00, pages 82–88, San Francisco, CA, USA, 2000. Morgan Kaufmann Publishers Inc. 

3.

4.

5.

6. 